<a href="https://colab.research.google.com/github/MarianaDuartee/ProjetoFinal/blob/main/spark_df_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
!pip install gcsfs

In [ ]:
!pip install pandera

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

from google.cloud import storage
from pyspark.sql.window import Window as W


import pyspark
import pyspark.sql.functions as F
import pandas as pd
import pandera as pa
import os
import gcsfs

In [ ]:
spark = SparkSession.builder\
.master('local')\
.appName('Projeto_Final')\
.config('spark.ui.enable', 'true')\
.config('spark.ui.port', '4050')\
.getOrCreate()

spark

serviceaccount = '/content/soulcode-projeto-final-4b88bea6e07a.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = serviceaccount

spark

In [ ]:
file_path = 'gs://data_lake_ingest_data/3_output/df_final_categorizado'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path):
    data = pd.read_csv(file_path, sep=',', encoding='UTF-8', header=0, index_col=False)

    # Validando esquema, coluna e tipo de dado
    schema = pa.DataFrameSchema(columns={"UF": pa.Column(str, pa.Check(str)),
                                         "Ano": pa.Column(int, nullable=True),
                                         "populacao_estimada": pa.Column(int,nullable=True),
                                         "Total_Ocorrencias": pa.Column(int, nullable=True),
                                         "Total_Vitimas": pa.Column(int, nullable=True),
                                         "Taxa_Analfabetismo": pa.Column(float, nullable=True),
                                         "Investimentos": pa.Column(float, nullable=True),
                                         "TotalOcorrencias_Roubos": pa.Column(int, nullable=True),
                                         "Media_Freq_Escolar": pa.Column(float, nullable=True),
                                         "TaxaDesemprego": pa.Column(float, nullable=True),
                                         "categoria_populacao": pa.Column(str, pa.Check(str)),
                                         "categoria_desemprego": pa.Column(str, pa.Check(str)),
                                         "categoria_ocorrencias": pa.Column(str, pa.Check(str)),
                                         "categoria_roubos": pa.Column(str, pa.Check(str)),                           
                                         })
    
    schema.validate(data)

df = spark.createDataFrame(data)
df.dtypes
df.show(10, truncate=False)

+----------+------------------+----+------------------+-----------------+-------------+------------------+-----------------+-----------------------+------------------+--------------+-------------------+--------------------+---------------------+----------------+
|Unnamed: 0|UF                |Ano |populacao_estimada|Total_Ocorrencias|Total_Vitimas|Taxa_Analfabetismo|Investimentos    |TotalOcorrencias_Roubos|Media_Freq_Escolar|TaxaDesemprego|categoria_populacao|categoria_desemprego|categoria_ocorrencias|categoria_roubos|
+----------+------------------+----+------------------+-----------------+-------------+------------------+-----------------+-----------------------+------------------+--------------+-------------------+--------------------+---------------------+----------------+
|0         |Piauí             |2016|3212180           |704              |704          |17.2              |8.0442947803E8   |5726                   |10.5              |9.43          |Pouco_populoso     |Medio_des

In [ ]:
df.select("UF", "Ano").filter(F.col("categoria_desemprego") == 'Baixo_desemprego').show()

+------------------+----+
|                UF| Ano|
+------------------+----+
|Mato Grosso do Sul|2016|
|    Santa Catarina|2016|
|          Rondônia|2016|
|    Santa Catarina|2017|
|    Santa Catarina|2018|
|Mato Grosso do Sul|2018|
|    Santa Catarina|2019|
|Mato Grosso do Sul|2019|
|          Rondônia|2019|
+------------------+----+



In [ ]:
df.select("UF", "Ano").filter(F.col("categoria_ocorrencias") == 'Pouco_perigoso').show()

+------------------+----+
|                UF| Ano|
+------------------+----+
|             Piauí|2016|
|Mato Grosso do Sul|2016|
|           Roraima|2016|
|              Acre|2016|
|             Amapá|2016|
|    Santa Catarina|2016|
|         Tocantins|2016|
|          Rondônia|2016|
|  Distrito Federal|2016|
|           Roraima|2017|
|             Piauí|2017|
|             Amapá|2017|
|          Rondônia|2017|
|              Acre|2017|
|Mato Grosso do Sul|2017|
|         Tocantins|2017|
|  Distrito Federal|2017|
|          Rondônia|2018|
|    Santa Catarina|2018|
|           Roraima|2018|
+------------------+----+
only showing top 20 rows



In [ ]:
df.select("UF", "Ano", "TaxaDesemprego").orderBy("TaxaDesemprego").show()

+------------------+----+--------------+
|                UF| Ano|TaxaDesemprego|
+------------------+----+--------------+
|    Santa Catarina|2019|          6.08|
|    Santa Catarina|2016|          6.32|
|    Santa Catarina|2018|           6.4|
|    Santa Catarina|2017|           7.1|
|Mato Grosso do Sul|2018|          7.55|
|Mato Grosso do Sul|2016|          7.68|
|          Rondônia|2016|          7.93|
|          Rondônia|2019|          7.95|
|Mato Grosso do Sul|2019|          7.95|
| Rio Grande do Sul|2019|          8.02|
| Rio Grande do Sul|2018|           8.1|
| Rio Grande do Sul|2016|          8.18|
|            Paraná|2016|          8.22|
|          Rondônia|2017|          8.23|
| Rio Grande do Sul|2017|           8.4|
|Mato Grosso do Sul|2017|          8.48|
|            Paraná|2019|          8.52|
|            Paraná|2018|          8.77|
|           Roraima|2016|           8.8|
|            Paraná|2017|           9.0|
+------------------+----+--------------+
only showing top

In [ ]:
df.select("UF", "Ano", "Total_Ocorrencias").orderBy("Total_Ocorrencias").show()

+------------------+----+-----------------+
|                UF| Ano|Total_Ocorrencias|
+------------------+----+-----------------+
|           Roraima|2016|              136|
|           Roraima|2017|              173|
|           Roraima|2019|              224|
|              Acre|2017|              231|
|             Amapá|2019|              270|
|             Amapá|2016|              292|
|              Acre|2019|              308|
|             Amapá|2017|              310|
|         Tocantins|2019|              326|
|           Roraima|2018|              350|
|             Amapá|2018|              352|
|         Tocantins|2017|              355|
|         Tocantins|2018|              386|
|  Distrito Federal|2019|              400|
|          Rondônia|2019|              401|
|              Acre|2018|              416|
|          Rondônia|2018|              424|
|              Acre|2016|              425|
|Mato Grosso do Sul|2019|              432|
|         Tocantins|2016|       

In [ ]:
df.select(F.col('UF'), F.col('Ano')).filter(F.col('Ano') > 2017).show()

+------------------+----+
|                UF| Ano|
+------------------+----+
|              Pará|2018|
|          Rondônia|2018|
|        Pernambuco|2018|
|      Minas Gerais|2018|
|    Santa Catarina|2018|
|           Paraíba|2018|
|             Ceará|2018|
|           Roraima|2018|
|    Espírito Santo|2018|
| Rio Grande do Sul|2018|
|  Distrito Federal|2018|
|            Paraná|2018|
|         São Paulo|2018|
|           Sergipe|2018|
|             Amapá|2018|
|Mato Grosso do Sul|2018|
|    Rio de Janeiro|2018|
|             Goiás|2018|
|           Alagoas|2018|
|             Bahia|2018|
+------------------+----+
only showing top 20 rows



In [ ]:
df.filter(df['UF'].rlike('C')).show()

+----------+--------------+----+------------------+-----------------+-------------+------------------+---------------+-----------------------+------------------+--------------+-------------------+--------------------+---------------------+----------------+
|Unnamed: 0|            UF| Ano|populacao_estimada|Total_Ocorrencias|Total_Vitimas|Taxa_Analfabetismo|  Investimentos|TotalOcorrencias_Roubos|Media_Freq_Escolar|TaxaDesemprego|categoria_populacao|categoria_desemprego|categoria_ocorrencias|categoria_roubos|
+----------+--------------+----+------------------+-----------------+-------------+------------------+---------------+-----------------------+------------------+--------------+-------------------+--------------------+---------------------+----------------+
|         7|         Ceará|2016|           8963663|             3285|         3407|              15.2|2.18831524988E9|                  15287|              10.6|         11.95|     Medio_populoso|    Medio_desemprego|            

In [ ]:
w1 = W.orderBy(F.col('Investimentos').desc())
df.withColumn("rank", F.rank().over(w1)).show()

+----------+-----------------+----+------------------+-----------------+-------------+------------------+-----------------+-----------------------+------------------+--------------+-------------------+--------------------+---------------------+----------------+----+
|Unnamed: 0|               UF| Ano|populacao_estimada|Total_Ocorrencias|Total_Vitimas|Taxa_Analfabetismo|    Investimentos|TotalOcorrencias_Roubos|Media_Freq_Escolar|TaxaDesemprego|categoria_populacao|categoria_desemprego|categoria_ocorrencias|categoria_roubos|rank|
+----------+-----------------+----+------------------+-----------------+-------------+------------------+-----------------+-----------------------+------------------+--------------+-------------------+--------------------+---------------------+----------------+----+
|        34|        São Paulo|2017|          45094866|             3677|         3894|               2.6|1.283378852809E10|                 183377|              12.1|          13.4|     Muito_populos

In [ ]:
w2 = W.orderBy(F.col('Total_Ocorrencias').desc())
df.withColumn("rank", F.rank().over(w2)).show()

+----------+--------------+----+------------------+-----------------+-------------+------------------+-----------------+-----------------------+------------------+--------------+-------------------+--------------------+---------------------+----------------+----+
|Unnamed: 0|            UF| Ano|populacao_estimada|Total_Ocorrencias|Total_Vitimas|Taxa_Analfabetismo|    Investimentos|TotalOcorrencias_Roubos|Media_Freq_Escolar|TaxaDesemprego|categoria_populacao|categoria_desemprego|categoria_ocorrencias|categoria_roubos|rank|
+----------+--------------+----+------------------+-----------------+-------------+------------------+-----------------+-----------------------+------------------+--------------+-------------------+--------------------+---------------------+----------------+----+
|        17|         Bahia|2016|          15276566|             6449|         6803|              13.0|  4.76059110341E9|                  21307|              10.4|         15.85|     Muito_populoso|     Alto_

In [ ]:
w3 = W.orderBy(F.col('populacao_estimada').desc())
df.withColumn("rank", F.rank().over(w3)).show()

+----------+-----------------+----+------------------+-----------------+-------------+------------------+-----------------+-----------------------+------------------+--------------+-------------------+--------------------+---------------------+----------------+----+
|Unnamed: 0|               UF| Ano|populacao_estimada|Total_Ocorrencias|Total_Vitimas|Taxa_Analfabetismo|    Investimentos|TotalOcorrencias_Roubos|Media_Freq_Escolar|TaxaDesemprego|categoria_populacao|categoria_desemprego|categoria_ocorrencias|categoria_roubos|rank|
+----------+-----------------+----+------------------+-----------------+-------------+------------------+-----------------+-----------------------+------------------+--------------+-------------------+--------------------+---------------------+----------------+----+
|        96|        São Paulo|2019|          45919049|             3068|         3200|               2.6|1.232196397253E10|                 137937|              12.3|         12.45|     Muito_populos

In [ ]:
#df.select(F.sum(F.col('Total_Ocorrencias'))).display()
df_temp = df.select(F.col('Total_Ocorrencias')).filter(F.col('Ano') == 2016)

df_temp = df_temp.withColumnRenamed('Total_Ocorrencias', 'Total_Ocorrencias_2016')

df_temp.select(F.sum(F.col('Total_Ocorrencias_2016'))).show()

+---------------------------+
|sum(Total_Ocorrencias_2016)|
+---------------------------+
|                      51441|
+---------------------------+



In [ ]:
df_temp_2 = df.select(F.col('Total_Ocorrencias')).filter(F.col('Ano') == 2017)

df_temp_2 = df_temp_2.withColumnRenamed('Total_Ocorrencias', 'Total_Ocorrencias_2017')

df_temp_2.select(F.sum(F.col('Total_Ocorrencias_2017'))).show()

+---------------------------+
|sum(Total_Ocorrencias_2017)|
+---------------------------+
|                      53759|
+---------------------------+



In [ ]:
df_temp_3 = df.select(F.col('Total_Ocorrencias')).filter(F.col('Ano') == 2018)

df_temp_3 = df_temp_3.withColumnRenamed('Total_Ocorrencias', 'Total_Ocorrencias_2018')

df_temp_3.select(F.sum(F.col('Total_Ocorrencias_2018'))).show()

+---------------------------+
|sum(Total_Ocorrencias_2018)|
+---------------------------+
|                      46421|
+---------------------------+



In [ ]:
df_temp_4 = df.select(F.col('Total_Ocorrencias')).filter(F.col('Ano') == 2019)

df_temp_4 = df_temp_4.withColumnRenamed('Total_Ocorrencias', 'Total_Ocorrencias_2019')

df_temp_4.select(F.sum(F.col('Total_Ocorrencias_2019'))).show()

+---------------------------+
|sum(Total_Ocorrencias_2019)|
+---------------------------+
|                      37866|
+---------------------------+



In [ ]:
df_temp_5 = df.select(F.col('Total_Ocorrencias')).filter(F.col('Ano') == 2016).filter(F.col('UF') == 'São Paulo')

df_temp_5 = df_temp_5.withColumnRenamed('Total_Ocorrencias', 'Total_Ocorrencias_2016_SP')

df_temp_5.select(F.sum(F.col('Total_Ocorrencias_2016_SP'))).show()

+------------------------------+
|sum(Total_Ocorrencias_2016_SP)|
+------------------------------+
|                          3885|
+------------------------------+



In [ ]:
df_temp_6 = df.select(F.col('Total_Ocorrencias')).filter(F.col('Ano') == 2017).filter(F.col('UF') == 'São Paulo')

df_temp_6 = df_temp_6.withColumnRenamed('Total_Ocorrencias', 'Total_Ocorrencias_2017_SP')

df_temp_6.select(F.sum(F.col('Total_Ocorrencias_2017_SP'))).show()

+------------------------------+
|sum(Total_Ocorrencias_2017_SP)|
+------------------------------+
|                          3677|
+------------------------------+



In [ ]:
df_temp_7 = df.select(F.col('Total_Ocorrencias')).filter(F.col('Ano') == 2018).filter(F.col('UF') == 'São Paulo')

df_temp_7 = df_temp_7.withColumnRenamed('Total_Ocorrencias', 'Total_Ocorrencias_2018_SP')

df_temp_7.select(F.sum(F.col('Total_Ocorrencias_2018_SP'))).show()

+------------------------------+
|sum(Total_Ocorrencias_2018_SP)|
+------------------------------+
|                          3299|
+------------------------------+



In [ ]:
df_temp_8 = df.select(F.col('Total_Ocorrencias')).filter(F.col('Ano') == 2019).filter(F.col('UF') == 'São Paulo')

df_temp_8 = df_temp_8.withColumnRenamed('Total_Ocorrencias', 'Total_Ocorrencias_2019_SP')

df_temp_8.select(F.sum(F.col('Total_Ocorrencias_2019_SP'))).show()

+------------------------------+
|sum(Total_Ocorrencias_2019_SP)|
+------------------------------+
|                          3068|
+------------------------------+

